In [2]:
import os
os.getcwd()


'/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/notebooks'

In [4]:
import os
os.listdir("../data_processed")


['R6jisatsunojoukyou_page32_table1_cleaned.csv',
 'master_suicide_data_aggregated.csv']

In [5]:
import pandas as pd

df = pd.read_csv("../data_processed/R6jisatsunojoukyou_page32_table1_cleaned.csv")
df.head()


,年,年齢層,人数
0,昭和53年,～９歳,866
1,昭和54年,～９歳,919
2,昭和55年,～９歳,678
3,昭和56年,～９歳,620
4,昭和57年,～９歳,599


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   年       517 non-null    object
 1   年齢層     517 non-null    object
 2   人数      428 non-null    object
dtypes: object(3)
memory usage: 12.2+ KB


In [8]:
import re

def era_to_year(val):
    # Extract era and number
    m = re.match(r"(令和|平成|昭和)(\d+)年", val)
    if not m:
        return None
    era, num = m.group(1), int(m.group(2))
    
    if era == "令和":
        return 2019 + (num - 1)
    if era == "平成":
        return 1989 + (num - 1)
    if era == "昭和":
        return 1926 + (num - 1)

df["year"] = df["年"].apply(era_to_year)
df[["年", "year"]].head()


,年,year
0,昭和53年,1978.0
1,昭和54年,1979.0
2,昭和55年,1980.0
3,昭和56年,1981.0
4,昭和57年,1982.0


In [9]:
df["年齢層"] = (
    df["年齢層"]
    .str.replace(" ", "", regex=False)
    .str.replace("　", "", regex=False)
    .str.replace("合計", "合計")
    .str.replace("合計", "合計")
    .str.replace("合計", "合計")
)

df["年齢層"].unique()


array(['～９歳', '10～19歳', '20～29歳', '30～39歳', '40～49歳', '50～59歳', '60～69歳',
       '70～79歳', '80歳～', '不詳', '合計'], dtype=object)

In [12]:
df["年齢層"] = df["年齢層"].str.replace("合計", "合計").str.replace("合", "合").str.replace("計", "計")
df["年齢層"] = df["年齢層"].str.replace("合計", "合計")


In [13]:
df["人数"] = (
    df["人数"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .astype(float)
)


In [14]:
df.sample(10)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年       517 non-null    object 
 1   年齢層     517 non-null    object 
 2   人数      428 non-null    float64
 3   year    495 non-null    float64
dtypes: float64(2), object(2)
memory usage: 16.3+ KB


In [15]:
df["年齢層"].unique()
df.sample(10)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   年       517 non-null    object 
 1   年齢層     517 non-null    object 
 2   人数      428 non-null    float64
 3   year    495 non-null    float64
dtypes: float64(2), object(2)
memory usage: 16.3+ KB


In [16]:
# 1. Show 年 values that failed conversion
df[df["year"].isna()][["年"]].drop_duplicates()

# 2. Show unique 年齢層 values
df["年齢層"].unique()

# 3. Show 人数 values that could not convert
bad = df[df["人数"].isna()][["年", "年齢層", "人数"]]
bad.head(20)


,年,年齢層,人数
47,昭和53年,10～19歳,NaN
48,昭和54年,10～19歳,NaN
49,昭和55年,10～19歳,NaN
50,昭和56年,10～19歳,NaN
51,昭和57年,10～19歳,NaN
52,昭和58年,10～19歳,NaN
53,昭和59年,10～19歳,NaN
54,昭和60年,10～19歳,NaN
55,昭和61年,10～19歳,NaN
56,昭和62年,10～19歳,NaN


In [17]:
df[df["year"].isna()]["年"].drop_duplicates().tolist()


['平成元年', '令和元年']

In [18]:
df["年齢層"].drop_duplicates().tolist()


['～９歳',
 '10～19歳',
 '20～29歳',
 '30～39歳',
 '40～49歳',
 '50～59歳',
 '60～69歳',
 '70～79歳',
 '80歳～',
 '不詳',
 '合計']

In [19]:
df.loc[47:70, ["人数"]]


,人数
47,NaN
48,NaN
49,NaN
50,NaN
51,NaN
52,NaN
53,NaN
54,NaN
55,NaN
56,NaN


In [21]:
# Fix era conversion properly

import re

def convert_japanese_era_to_year(era_str):
    # Normalize full-width digits
    era_str = era_str.replace("元年", "1年")

    patterns = {
        "令和": 2018,
        "平成": 1988,
        "昭和": 1925,
        "大正": 1911,
        "明治": 1867
    }

    for era, base in patterns.items():
        if era in era_str:
            m = re.search(r"(\d+)年", era_str)
            if m:
                year_num = int(m.group(1))
                return base + year_num
    return None


In [22]:
df["year"] = df["年"].apply(convert_japanese_era_to_year)


In [23]:
df["年齢層"].drop_duplicates().tolist()


['～９歳',
 '10～19歳',
 '20～29歳',
 '30～39歳',
 '40～49歳',
 '50～59歳',
 '60～69歳',
 '70～79歳',
 '80歳～',
 '不詳',
 '合計']

In [24]:
age_map = {
    "～９歳": "0～9歳",
    "10～19歳": "10～19歳",
    "20～29歳": "20～29歳",
    "30～39歳": "30～39歳",
    "40～49歳": "40～49歳",
    "50～59歳": "50～59歳",
    "60～69歳": "60～69歳",
    "70～79歳": "70～79歳",
    "80歳～": "80歳以上",
    "不詳": "不詳",
    "合計": "合計"
}

df["年齢層"] = df["年齢層"].str.strip().replace(age_map)


In [25]:
df.loc[47:70, ["人数"]]


,人数
47,NaN
48,NaN
49,NaN
50,NaN
51,NaN
52,NaN
53,NaN
54,NaN
55,NaN
56,NaN


In [28]:
df.to_csv("age_cleaned.csv", index=False)

In [31]:
import pandas as pd

page33 = pd.read_csv("../data_raw/R6jisatsunojoukyou_page33_table1.csv")
page31 = pd.read_csv("../data_raw/R6jisatsunojoukyou_page31_table1.csv")
page30_table3 = pd.read_csv("../data_raw/R6jisatsunojoukyou_page30_table3.csv")
page30_table4 = pd.read_csv("../data_raw/R6jisatsunojoukyou_page30_table4.csv")

page33.head(), page31.head(), page30_table3.head(), page30_table4.head()



FileNotFoundError: [Errno 2] No such file or directory: '../data_raw/R6jisatsunojoukyou_page33_table1.csv'

In [33]:
import pandas as pd
import os

PROCESSED_DIR = "/home/yulia-chekhovska/Public-Health-Data-Engineering-Project-Japan-Suicide-Statistics-Pipeline/data_processed/"

# ======================================================================
# Helper: convert Japanese eras (e.g., 昭和, 平成, 令和) to Western year
# ======================================================================

def convert_japanese_year(era_string):
    if pd.isna(era_string):
        return None

    era_string = str(era_string)

    if era_string.startswith("昭和"):
        year = int(era_string.replace("昭和", "").replace("年", ""))
        return 1925 + year
    if era_string.startswith("平成"):
        year = era_string.replace("平成", "").replace("年", "")
        year = 1 if year == "元" else int(year)
        return 1988 + year
    if era_string.startswith("令和"):
        year = era_string.replace("令和", "").replace("年", "")
        year = 1 if year == "元" else int(year)
        return 2018 + year

    return None


# ======================================================================
# CLEAN page31 and page33 (simple year + age structure)
# ======================================================================

def clean_simple_table(df, name):
    df = df.copy()

    # First column is year
    df.rename(columns={df.columns[0]: "年"}, inplace=True)

    # Convert era year to Western
    df["year"] = df["年"].apply(convert_japanese_year)

    # Melt to long format
    df_long = df.melt(id_vars=["年", "year"], var_name="カテゴリ", value_name="人数")

    # Clean numbers
    df_long["人数"] = (
        df_long["人数"]
        .astype(str)
        .str.replace(",", "", regex=False)
        .str.replace(" ", "", regex=False)
    )
    df_long["人数"] = pd.to_numeric(df_long["人数"], errors="coerce")

    out_path = os.path.join(PROCESSED_DIR, f"{name}_cleaned.csv")
    df_long.to_csv(out_path, index=False, encoding="utf-8-sig")

    print(f"Saved cleaned table: {out_path}")
    return df_long


# ======================================================================
# CLEAN page30 tables (problem categories)
# ======================================================================

def clean_problem_table(df, name):
    df = df.copy()

    df.rename(columns={"Unnamed: 0": "年"}, inplace=True)
    df["year"] = df["年"].apply(convert_japanese_year)

    # Standardize column names
    df.columns = [c.replace("\n", "").strip() for c in df.columns]

    df_long = df.melt(id_vars=["年", "year"], var_name="問題分類", value_name="人数")

    df_long["人数"] = (
        df_long["人数"]
        .astype(str)
        .str.replace(",", "", regex=False)
    )
    df_long["人数"] = pd.to_numeric(df_long["人数"], errors="coerce")

    out_path = os.path.join(PROCESSED_DIR, f"{name}_cleaned.csv")
    df_long.to_csv(out_path, index=False, encoding="utf-8-sig")

    print(f"Saved cleaned problem table: {out_path}")
    return df_long
